In [1]:
import getGmail_Class
import os, sys, logging
from tqdm import trange
from datetime import datetime
import pandas as pd
import re

In [2]:
gGC = getGmail_Class.Email()

In [3]:
Num, Name, investment_company, Date, Path = [[] for i in range(5)]

# Create log file
if not os.path.isdir("log"):
    os.mkdir("log")

FORMAT = '%(asctime)s %(levelname)s: %(message)s'
logging.basicConfig(level = logging.INFO, filename = "./log/" + datetime.now().strftime("%Y_%m_%d") + '.log', filemode = 'w', format = FORMAT)
logging.info('Updating email start')

# request a list of all the messages
result = gGC.service.users().messages().list(userId = 'me', maxResults = 500, labelIds = ["INBOX"]).execute()
messages = result.get('messages')

if len(messages) == 0:
    logging.info('Inbox quantity is 0')
    logging.info('Updating email end')
    sys.exit(0)

# iterate through all the messages
for i in trange(len(messages)):
    # Get the message from its id
    txt = gGC.service.users().messages().get(userId = 'me', id = messages[i]['id']).execute()
    payload = txt['payload']
    headers = payload['headers']
    date = gGC.getDate(headers)
    subject = gGC.verifySubject(headers)

    if len(subject) != 0:
        tempNum, tempName, tempInvestment_company, tempPath = gGC.getResearch_report(subject, payload, messages[i]['id'])

        if len(tempNum) != 0:
            for j in range(len(tempNum)):
                Num.append(tempNum[j])
                Name.append(tempName[j])
                Date.append(date)
                investment_company.append(tempInvestment_company)
                Path.append(tempPath[j])
#                     # Modify labels
#                     gGC.modifyLabels(messages[i]['id'], "Label2")
    else:
        # Modify labels
        gGC.modifyLabels(messages[i]['id'], "Label3")
        i = i

df = pd.DataFrame({ "Number" : Num, "Name" : Name, "Investment company" : investment_company, "Date" : Date, "File path" : Path })

logging.info('Updating email end')

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [20:07<00:00,  2.41s/it]


In [4]:
df

,Number,Name,Investment company,Date,File path
0,2892,第一金,,2022_3_03,./file/2892/統一投顧-陳晏平-【投資速報】第一金 (2892.TT，中立) 營運...
1,2892,第一金,,2021_12_01,./file/2892/統一投顧-陳晏平-【投資速報】第一金 (2892.TT，中立) 獲利...
2,2892,第一金,,2021_6_01,./file/2892/統一投顧-陳晏平-【投資速報】第一金 (2892.TT，中立) 20...
3,2887,台新金,,2021_5_20,./file/2887/統一投顧-陳晏平-【投資速報】台新金 (2887.TT，中立) 獲利...
4,2892,第一金,,2020_12_01,./file/2892/統一投顧-陳晏平-【投資速報】第一金 (2892.TT，中立) 獲利...
...,...,...,...,...,...
264,2308,台達電,,2020_7_31,./file/2308/台達電(2308)-20200731-速報.pdf
265,2301,光寶科,,2020_7_31,./file/2301/光寶科(2301)-20200731-速報.pdf
266,2367,燿華,,2020_7_31,./file/2367/燿華(2367)-20200731-速報.pdf
267,4966,譜瑞-KY,,2020_7_31,./file/4966/譜瑞-KY(4966)-20200731-速報.pdf
